# Evaluation of SIFT positional angular uncertatinty
0) Specify datasets

In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

from sift_uncertainty import *

scenes = ['Alamo', 'Ellis_Island', 'Madrid_Metropolis', 'NYC_Library', 'Piazza_del_Popolo', 'Roman_Forum', 'Tower_of_London', 'Union_Square', 'Vienna_Cathedral', 'Yorkminster']
scales = [3.97745, 0.94967, 14.44161, 6.95844, 6.25074, 24.38904, 15.72399, 6.85125, 15.07437, 13.26471]

1) Load all the keypoints (angles, scales, positions) and GT homographies

In [ ]:
data = {'sift_a1':[], 'sift_a2': [], 'sift_s1': [], 'sift_s2': [], 'sift_u1': [], 'sift_u2': [], 'gt_H1to2': [], 'gt_Hids': []}
for scene, scale in zip(scenes, scales):
    data = collect_all(scene, scale, data)

2. Calculate the SIFT angular transformations

In [ ]:
sift_a12 = np.array(data['sift_a2']) - np.array(data['sift_a1'])
sift_a12[sift_a12 > 180] = sift_a12[sift_a12 > 180] - 360
sift_a12[sift_a12 < -180] = sift_a12[sift_a12 < -180] + 360

plt.hist(sift_a12, 90)
plt.xlabel(r'The detector angular transformation $\alpha_i$')
plt.ylabel('Occurrence')
plt.title('Histogram of the detector angular transformation')
plt.yscale('log')
plt.savefig('histogram_of_detector_angular_transformation2.png', dpi=100)
plt.show()

3) Aproximate the $H_i$ localy by the affinity matrix $A_i$

In [ ]:
ACs = [[] for i in range(len(data['sift_a1']))]
H2pts_ids = [[] for j in range(len(data['gt_Hids']))]
for j,x in enumerate(data['gt_Hids']):
    H2pts_ids[x].append(j)

for i in tqdm(range(len(data['gt_H1to2']))):
    ids = H2pts_ids[i]
    H = data['gt_H1to2'][i]
    u1 = np.array([data['sift_u1'][j] for j in ids])
    u2 = np.array([data['sift_u2'][j] for j in ids])
    ACs_arr = affines_from_homography_wf2(H, u1)     #, u2
    for j, id in enumerate(ids):
        ACs[id] = ACs_arr[j,:,:]

4) Calculate the angular transformation from the projection of directional vectors

In [ ]:
a1 = np.array(data['sift_a1']) * np.pi / 180 
a2 = np.array(data['sift_a2']) * np.pi / 180 
sift_d1 = np.array([np.cos(a1), np.sin(a1)])
sift_d2 = np.array([np.cos(a2), np.sin(a2)])

A_d1 = np.array([ACs[id] @ sift_d1[:,id] for id in range(len(data['sift_a1']))]).T
d_a12_Adirect = directions_to_differential_angle(sift_d2, A_d1) * 180 / np.pi

sift_d10 = np.concatenate([sift_d1, np.zeros((1,sift_d1.shape[1]))], axis=0)
H_d1 = np.array([data['gt_H1to2'][data['gt_Hids'][id]] @ sift_d10[:,id] for id in range(len(data['sift_a1']))]).T
d_a12_Hdirect = directions_to_differential_angle(sift_d2, H_d1[0:2,:]) * 180 / np.pi

5) Decompose $A_i$ it to the angles by QT, SVD, and exponential analysis

In [ ]:
gt_a12_svd, gt_a12_logm, gt_s12, cond_num_svd, shear_mag_logm = decompose_affines(ACs)

6) Calculate the $\Delta \alpha$ for different decompositions

In [ ]:
d_a12_svd = (np.array(gt_a12_svd)*180/np.pi) - sift_a12
d_a12_logm = (np.array(gt_a12_logm)*180/np.pi) - sift_a12

7) Filter out the angular transformations with condition number > 1.2

In [ ]:
cond_num_filter = np.array(cond_num_svd) < 1.2
d_a12_Adirect_filtered = d_a12_Adirect[cond_num_filter]
d_a12_svd_filtered = d_a12_svd[cond_num_filter]
d_a12_logm_filtered = d_a12_logm[cond_num_filter]

8) Plot the histogram of main approaches (directional, SVD, logm) angular transformation errors

In [ ]:
plt.hist(sift_a12, 90, label=r'$\alpha$')
space = np.linspace(-180, 180, 90)
h1, edges1 = np.histogram(d_a12_Adirect_filtered, bins=space)
plt.stairs(h1, edges1, label=r'$\Delta \alpha_{direct}$')
h2, edges2 = np.histogram(d_a12_svd_filtered, bins=space)
plt.stairs(h2, edges2, label=r'$\Delta \alpha_{SVD}$')
h3, edges3 = np.histogram(d_a12_logm_filtered, bins=space)
plt.stairs(h3, edges3, label=r'$\Delta \alpha_{logm}$')
plt.title('Histogram of the angular transformation error') 
plt.legend(loc='upper right')
plt.xlabel(r'The angular transformation error $\Delta \alpha$')
plt.ylabel('Occurrence (log-scale)')
plt.yscale('log')
plt.savefig('log_histogram_of_angular_transformation_error.png', dpi=100)
plt.show()

In [ ]:
print(f'The standard deviation of angular transformation error: {np.std(d_a12_logm_filtered)} deg.')
print(f'The number of samples that fulfill the condition number threshold: {np.shape(d_a12_logm_filtered)[0]} out of {np.shape(cond_num_svd)[0]} correspondences.')

BONUS: evaluation of average error introduced by sheers for given condition number of A_i 

In [ ]:
p0_angle = -13.9462
p1_angle = 14.5756
p0_scale = -39.6470
p1_scale = 39.2295
edges_mean = (edges[0:-1] + edges[1:])/2
angle_sheer_err = p0_angle + edges_mean * p1_angle
scale_sheer_err = p0_scale + edges_mean * p1_scale
angle_sheer_err_integral = []
scale_sheer_err_integral = []
for i in range(len(angle_sheer_err)):
    weights = h[0:i+1] / np.sum(h[0:i+1])
    angle_sheer_err_integral.append(np.sum(angle_sheer_err[0:i+1] * weights))
    scale_sheer_err_integral.append(np.sum(scale_sheer_err[0:i+1] * weights))
plt.scatter(edges_mean[1:], angle_sheer_err_integral[1:], label=r'average angular error [deg]')
plt.scatter(edges_mean[1:], scale_sheer_err_integral[1:], label=r'average scale error [%]')
plt.title('Histogram of the average error introduced by sheers: cond($A_i$) < threshold') 
plt.legend(loc='upper left')
plt.xlabel(r'Threshold for condition number')
plt.ylabel('Average error introduced by sheers')
plt.savefig('histogram_of_cond_num2error_integral.png', dpi=100)
plt.show()
